# 📚 Modular Literature Finder

**Multi-database academic search with API key fallback**

## Supported Databases

| Database | API Key | Rate Limit | Best For |
|----------|---------|------------|----------|
| PubMed | ❌ | Unlimited | 의학/생명과학 |
| arXiv | ❌ | Unlimited | CS/물리/수학 |
| OSF | ❌ | Unlimited | 사회과학/Preprints |
| GitHub | ⚠️ 권장 | 60/hr (free) | 코드/도구 |
| Semantic Scholar | ⚠️ 권장 | 100/5min | 통합 검색 |

---

## 1. Setup

In [ ]:
# Install dependencies
!pip install -q requests google-generativeai

import requests
import json
import time
import re
from dataclasses import dataclass, field
from typing import List, Optional, Dict
from abc import ABC, abstractmethod
from getpass import getpass

print("✓ Setup complete")

## 2. API Key Configuration (Optional)

**모두 선택사항입니다!** 없어도 기본 검색은 동작합니다.

In [ ]:
# API Keys (모두 선택사항 - 없으면 Enter)
print("API Keys 입력 (없으면 Enter를 누르세요)")
print("-" * 40)

GEMINI_API_KEY = getpass("Gemini API Key (LLM용): ") or None
GITHUB_TOKEN = getpass("GitHub Token (선택): ") or None
SEMANTIC_SCHOLAR_KEY = getpass("Semantic Scholar Key (선택): ") or None

# Status
print("\n📊 API Key Status:")
print(f"  Gemini: {'✓ 설정됨' if GEMINI_API_KEY else '✗ 없음 (요약 기능 제한)'}")
print(f"  GitHub: {'✓ 설정됨' if GITHUB_TOKEN else '⚠️ 없음 (60 req/hr 제한)'}")
print(f"  Semantic Scholar: {'✓ 설정됨' if SEMANTIC_SCHOLAR_KEY else '⚠️ 없음 (rate limited)'}")
print("\n✓ PubMed, arXiv, OSF는 API key 불필요!")

## 3. Data Structure

In [ ]:
@dataclass
class Paper:
    """Unified paper format across all databases"""
    title: str
    authors: List[str]
    year: Optional[int]
    abstract: str
    url: str
    source: str  # 'pubmed', 'arxiv', 'osf', 'github', 'semantic_scholar'
    doi: Optional[str] = None
    citations: Optional[int] = None
    keywords: List[str] = field(default_factory=list)

    def __repr__(self):
        author_str = self.authors[0] if self.authors else "Unknown"
        if len(self.authors) > 1:
            author_str += " et al."
        return f"[{self.source.upper()}] {author_str} ({self.year}): {self.title[:60]}..."

print("✓ Paper dataclass defined")

## 4. Database Modules

각 모듈은 독립적으로 동작하며, 순서 변경/추가/제거 가능

In [ ]:
class BaseModule(ABC):
    """Base class for all database modules"""
    name: str = "base"
    requires_api_key: bool = False

    @abstractmethod
    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        pass

    def is_available(self) -> bool:
        return True

print("✓ BaseModule defined")

In [ ]:
class PubMedModule(BaseModule):
    """PubMed/NCBI E-utilities - 의학/생명과학"""
    name = "pubmed"
    requires_api_key = False

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []

        # Step 1: Search for IDs
        search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        search_params = {
            "db": "pubmed",
            "term": query,
            "retmax": max_results,
            "retmode": "json"
        }

        try:
            resp = requests.get(search_url, params=search_params, timeout=10)
            ids = resp.json().get("esearchresult", {}).get("idlist", [])

            if not ids:
                return papers

            # Step 2: Fetch details
            fetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi"
            fetch_params = {
                "db": "pubmed",
                "id": ",".join(ids),
                "retmode": "json"
            }

            resp = requests.get(fetch_url, params=fetch_params, timeout=10)
            results = resp.json().get("result", {})

            for pmid in ids:
                if pmid in results:
                    item = results[pmid]
                    authors = [a.get("name", "") for a in item.get("authors", [])]

                    papers.append(Paper(
                        title=item.get("title", ""),
                        authors=authors[:5],
                        year=int(item.get("pubdate", "0")[:4]) if item.get("pubdate") else None,
                        abstract="",  # Summary doesn't include abstract
                        url=f"https://pubmed.ncbi.nlm.nih.gov/{pmid}/",
                        source="pubmed",
                        doi=item.get("elocationid", "").replace("doi: ", "")
                    ))

        except Exception as e:
            print(f"  ⚠️ PubMed error: {e}")

        return papers

print("✓ PubMedModule defined")

In [ ]:
class ArXivModule(BaseModule):
    """arXiv API - CS/물리/수학"""
    name = "arxiv"
    requires_api_key = False

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []
        url = "http://export.arxiv.org/api/query"
        params = {
            "search_query": f"all:{query}",
            "start": 0,
            "max_results": max_results
        }

        try:
            resp = requests.get(url, params=params, timeout=10)
            # Parse XML (simple regex for demo)
            entries = re.findall(r'<entry>(.*?)</entry>', resp.text, re.DOTALL)

            for entry in entries:
                title = re.search(r'<title>(.*?)</title>', entry, re.DOTALL)
                abstract = re.search(r'<summary>(.*?)</summary>', entry, re.DOTALL)
                authors = re.findall(r'<name>(.*?)</name>', entry)
                link = re.search(r'<id>(.*?)</id>', entry)
                published = re.search(r'<published>(.*?)</published>', entry)

                if title and link:
                    year = int(published.group(1)[:4]) if published else None
                    papers.append(Paper(
                        title=title.group(1).strip().replace('\n', ' '),
                        authors=authors[:5],
                        year=year,
                        abstract=abstract.group(1).strip()[:500] if abstract else "",
                        url=link.group(1),
                        source="arxiv"
                    ))

        except Exception as e:
            print(f"  ⚠️ arXiv error: {e}")

        return papers

print("✓ ArXivModule defined")

In [ ]:
class OSFModule(BaseModule):
    """OSF (Open Science Framework) - 사회과학/Preprints"""
    name = "osf"
    requires_api_key = False

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []
        url = "https://api.osf.io/v2/preprints/"
        params = {
            "filter[title,description]": query,
            "page[size]": max_results
        }

        try:
            resp = requests.get(url, params=params, timeout=10)
            data = resp.json().get("data", [])

            for item in data:
                attrs = item.get("attributes", {})
                date_created = attrs.get("date_created", "")

                papers.append(Paper(
                    title=attrs.get("title", ""),
                    authors=[],  # OSF API doesn't return authors in list
                    year=int(date_created[:4]) if date_created else None,
                    abstract=attrs.get("description", "")[:500],
                    url=item.get("links", {}).get("html", ""),
                    source="osf",
                    doi=attrs.get("doi", "")
                ))

        except Exception as e:
            print(f"  ⚠️ OSF error: {e}")

        return papers

print("✓ OSFModule defined")

In [ ]:
class GitHubModule(BaseModule):
    """GitHub Search - 코드/도구/Awesome Lists"""
    name = "github"
    requires_api_key = False  # Works without, but rate limited

    def __init__(self, token: Optional[str] = None):
        self.token = token

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []
        url = "https://api.github.com/search/repositories"

        headers = {"Accept": "application/vnd.github.v3+json"}
        if self.token:
            headers["Authorization"] = f"token {self.token}"

        params = {
            "q": f"{query} in:readme,description",
            "sort": "stars",
            "per_page": max_results
        }

        try:
            resp = requests.get(url, headers=headers, params=params, timeout=10)

            if resp.status_code == 403:
                print("  ⚠️ GitHub rate limit hit")
                return papers

            items = resp.json().get("items", [])

            for item in items:
                created = item.get("created_at", "")
                papers.append(Paper(
                    title=item.get("full_name", ""),
                    authors=[item.get("owner", {}).get("login", "")],
                    year=int(created[:4]) if created else None,
                    abstract=item.get("description", "") or "",
                    url=item.get("html_url", ""),
                    source="github",
                    citations=item.get("stargazers_count", 0),  # Stars as proxy
                    keywords=item.get("topics", [])
                ))

        except Exception as e:
            print(f"  ⚠️ GitHub error: {e}")

        return papers

print("✓ GitHubModule defined")

In [ ]:
class SemanticScholarModule(BaseModule):
    """Semantic Scholar API - 통합 학술 검색"""
    name = "semantic_scholar"
    requires_api_key = False  # Works without, but rate limited

    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []
        url = "https://api.semanticscholar.org/graph/v1/paper/search"

        headers = {}
        if self.api_key:
            headers["x-api-key"] = self.api_key

        params = {
            "query": query,
            "limit": max_results,
            "fields": "title,authors,year,abstract,url,citationCount,externalIds"
        }

        try:
            resp = requests.get(url, headers=headers, params=params, timeout=10)

            if resp.status_code == 429:
                print("  ⚠️ Semantic Scholar rate limit - waiting 5s...")
                time.sleep(5)
                resp = requests.get(url, headers=headers, params=params, timeout=10)

            data = resp.json().get("data", [])

            for item in data:
                authors = [a.get("name", "") for a in item.get("authors", [])]
                ext_ids = item.get("externalIds", {}) or {}

                papers.append(Paper(
                    title=item.get("title", ""),
                    authors=authors[:5],
                    year=item.get("year"),
                    abstract=item.get("abstract", "")[:500] if item.get("abstract") else "",
                    url=item.get("url", ""),
                    source="semantic_scholar",
                    doi=ext_ids.get("DOI", ""),
                    citations=item.get("citationCount")
                ))

        except Exception as e:
            print(f"  ⚠️ Semantic Scholar error: {e}")

        return papers

print("✓ SemanticScholarModule defined")

## 5. Literature Pipeline

모듈들을 조합하여 통합 검색 파이프라인 구성

In [ ]:
class LiteraturePipeline:
    """Modular literature search pipeline"""

    def __init__(self, modules: List[BaseModule] = None):
        self.modules = modules or []

    def add(self, module: BaseModule):
        """Add a module to the pipeline"""
        self.modules.append(module)
        return self

    def remove(self, module_name: str):
        """Remove a module by name"""
        self.modules = [m for m in self.modules if m.name != module_name]
        return self

    def reorder(self, order: List[str]):
        """Reorder modules by name list"""
        name_to_module = {m.name: m for m in self.modules}
        self.modules = [name_to_module[n] for n in order if n in name_to_module]
        return self

    def list_modules(self):
        """Show current modules"""
        for i, m in enumerate(self.modules, 1):
            key_status = "🔑" if m.requires_api_key else "✓"
            print(f"  {i}. {m.name} {key_status}")

    def search(self, query: str, max_per_source: int = 5) -> List[Paper]:
        """Search across all modules"""
        all_papers = []

        print(f"\n🔍 Searching: '{query}'")
        print("=" * 50)

        for module in self.modules:
            print(f"\n📚 {module.name.upper()}...")

            if not module.is_available():
                print(f"  ⏭️ Skipped (not available)")
                continue

            papers = module.search(query, max_per_source)
            all_papers.extend(papers)
            print(f"  ✓ Found {len(papers)} results")

            time.sleep(0.5)  # Be nice to APIs

        # Deduplicate by title similarity
        unique_papers = self._deduplicate(all_papers)

        print("\n" + "=" * 50)
        print(f"📊 Total: {len(unique_papers)} unique papers")

        return unique_papers

    def _deduplicate(self, papers: List[Paper]) -> List[Paper]:
        """Remove duplicates based on title similarity"""
        seen_titles = set()
        unique = []

        for paper in papers:
            # Normalize title for comparison
            norm_title = paper.title.lower().strip()[:50]
            if norm_title not in seen_titles:
                seen_titles.add(norm_title)
                unique.append(paper)

        return unique

print("✓ LiteraturePipeline defined")

## 6. Create Your Pipeline

원하는 모듈만 선택하여 파이프라인 구성

In [ ]:
# Create pipeline with selected modules
pipeline = LiteraturePipeline([
    PubMedModule(),                              # 의학/생명과학
    ArXivModule(),                               # CS/물리/수학
    OSFModule(),                                 # 사회과학
    GitHubModule(token=GITHUB_TOKEN),            # 코드/도구
    SemanticScholarModule(api_key=SEMANTIC_SCHOLAR_KEY),  # 통합 검색
])

print("📋 Current Pipeline:")
pipeline.list_modules()

In [ ]:
# Optional: Customize pipeline

# Remove a module
# pipeline.remove("github")

# Reorder modules
# pipeline.reorder(["semantic_scholar", "pubmed", "arxiv"])

# Add custom module
# pipeline.add(MyCustomModule())

print("📋 Modified Pipeline:")
pipeline.list_modules()

## 7. Run Search

In [ ]:
# Your research query
QUERY = "large language models emotion recognition"  # @param {type:"string"}

# Run search
results = pipeline.search(QUERY, max_per_source=5)

In [ ]:
# Display results
print("\n" + "=" * 70)
print("SEARCH RESULTS")
print("=" * 70)

for i, paper in enumerate(results[:20], 1):
    print(f"\n{i}. [{paper.source.upper()}] {paper.title[:70]}")
    if paper.authors:
        print(f"   Authors: {', '.join(paper.authors[:3])}")
    if paper.year:
        print(f"   Year: {paper.year}")
    if paper.citations:
        print(f"   Citations: {paper.citations}")
    print(f"   URL: {paper.url}")

## 8. Export Results

In [ ]:
# Export to JSON
export_data = [
    {
        "title": p.title,
        "authors": p.authors,
        "year": p.year,
        "source": p.source,
        "url": p.url,
        "doi": p.doi,
        "citations": p.citations
    }
    for p in results
]

with open("literature_results.json", "w") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

# Export to markdown
md_content = f"# Literature Search: {QUERY}\n\n"
for i, p in enumerate(results, 1):
    authors = ", ".join(p.authors[:3]) if p.authors else "Unknown"
    md_content += f"{i}. **{p.title}**\n"
    md_content += f"   - {authors} ({p.year or 'n.d.'})\n"
    md_content += f"   - Source: {p.source} | [Link]({p.url})\n\n"

with open("literature_results.md", "w") as f:
    f.write(md_content)

print("✓ Exported to:")
print("  - literature_results.json")
print("  - literature_results.md")

# Download (Colab)
try:
    from google.colab import files
    files.download("literature_results.json")
    files.download("literature_results.md")
except:
    pass

---

## 9. Create Your Own Module (Template)

새로운 데이터베이스 모듈을 만들어보세요!

In [ ]:
class MyCustomModule(BaseModule):
    """Template for creating custom database module"""
    name = "my_custom"
    requires_api_key = False

    def __init__(self, api_key: Optional[str] = None):
        self.api_key = api_key

    def search(self, query: str, max_results: int = 10) -> List[Paper]:
        papers = []

        # TODO: Implement your search logic here
        # 1. Call your API
        # 2. Parse response
        # 3. Create Paper objects
        # 4. Return list of Papers

        # Example:
        # resp = requests.get("https://api.example.com/search", params={"q": query})
        # for item in resp.json()["results"]:
        #     papers.append(Paper(
        #         title=item["title"],
        #         authors=item["authors"],
        #         year=item["year"],
        #         abstract=item["abstract"],
        #         url=item["url"],
        #         source="my_custom"
        #     ))

        return papers

# Usage:
# pipeline.add(MyCustomModule())

---

## API Reference

| Database | API Docs | Rate Limit |
|----------|----------|------------|
| PubMed | [E-utilities](https://www.ncbi.nlm.nih.gov/books/NBK25500/) | No limit |
| arXiv | [arXiv API](https://arxiv.org/help/api/index) | No limit |
| OSF | [OSF API](https://developer.osf.io/) | No limit |
| GitHub | [GitHub API](https://docs.github.com/en/rest) | 60/hr (free) |
| Semantic Scholar | [S2 API](https://api.semanticscholar.org/) | 100/5min |

---

*SNU AI Psychology Workshop - February 2026*